data 변수 => 비식별 전<br>
oriental 변수 => 비식별은 됐고, K적용 안된것<br>
same_size 변수 => 비식별 됐고, K적용 안된걸 0으로 바꾼 데이터 ==> 개수를 맞춰야해서 짤린거 0으로 채움<br>
after 변수 => 최종본<br>

유용성 테스트는 data랑 same_size랑 준식별자만 비교

In [1]:
import sys
import pandas as pd #import csv
import numpy as np
#from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QGridLayout, QFileDialog, QLabel, QStackedWidget
from PyQt5.QtCore import *
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *
from PyQt5 import uic, QtGui
import math
from itertools import product
from numpy import dot
from numpy.linalg import norm

global type_to # 타입 변경을 위한 변수
global type_combo_res
global data # 비식별 전
global count # 데이터셋 중 수치형 데이터의 개수
global info_header
global preview
global top3
global level_tmp # 산출한 level들을 넣을 list
global after

preview = 10 # 프리뷰로 보여줄 데이터 개수. 기존의 len(data)
info_header = ['이름', '최솟값', '최댓값', '평균', '중앙값', '표준편차', '최빈값'] # 수치형 info
type_combo_res = []
level_tmp = []

In [2]:
# 비식별 기법
class anonymous() :
    def __init__(self,df):
        self.df = df
        self.backup = backup
    
    def delete_col(self, column):
        """
            delete_col : 열 삭제
            column : 삭제할 column명
        """
        self.backup = self[column]
        
        for i in range(len(self[column])):
            self[column].iloc[i] = "" # 공백으로 값 변경
            
        return after
    
    def delete_row(self, rows):
        """
            delete_row : 행 삭제
            - rows : 삭제할 row명
        """
        self.backup = self.iloc[rows]
        del_df = self.drop(rows, axis=0)
        return del_df
    
    def delete_partial(self, column, num, sep):
        data = self[column] 

        try :
            a = list(map(lambda x : sep.join(x.split(sep)[i] for i in range(num)),data))
            self[column] = a
            after = self
            return after
        except IndexError :
            return ""
        
    # (종근) 마스킹 수정
    def masking(self,column,direction,number, max_len): # DF, column, 남겨둘 정보 위치(left/right), 남겨둘 정보 개수, column 내 데이터 중 가장 긴 길이
        self.backup = self[column]
        before_data=self.loc[:,column]   
        after = self.astype({column : 'object'})           #columns 데이터 타입 변경
        
        for i in range(len(before_data.index)):
            string = str(before_data.iloc[i])
            length = len(string)                  
            m_length = length - number            #마스킹할 문자 길이 구하기
            if(max_len>length):
                string = string +(" "*(max_len-length))
            length = len(string)
            
            if direction == 'left':             #남겨둘 정보(마스킹하지 않을 정보)의 위치=왼쪽
                if(length<number):
                    string ="*"*len(string)
                else:
                    string = string[:number]+'*'*len(string[number:])

            after.at[i,column] = string

        return after 
    
    # (민석) 상하단 코딩 수정
    def Drange(self,column,num):   #민석스 버전 drange      #(데이터프레임, 적용할 칼럼, 수의 간격)
        self.backup = self[column]
        before_data = self.loc[:,column]
        re_cut = pd.cut(before_data, num)
        after = self.astype({column : 'category'})
        self[column] = re_cut
        after = self
        return after

    def Rounding(self,column,loc,job):  #데이터프레임, 컬럼, 위치, 작업        
        if job==0:#반올림            
            self[column] = round(self[column], -loc)
            after=self
        return after
    
    def Pretreatment(self,column): # count =1 인 값 제거
        list2=[]
        list3=self[column].tolist()
        for i in list3:
            if(list3.count(i)==1):
                i = ''
                list2.append(i)
            else:
                list2.append(i)
        list3=list2
        self[column]=list3
        after=self
         
    def object_normalize(self,column): #문자형 데이터 전처리
        s = pd.Series(self[column]).value_counts()
        s = (s-s.mean())/s.std()
        l_overvalue =s[s<-3].index
        result = l_overvalue
        for i in result:
            after= self.loc[self[column]!=i]
    
    #일반화
    def generation(self,column,level,delimeter) : # 구분자 delimeter
        listing = self[column] #레벨은 역순임. Level1이 제일 쌘 비식별이 됨
        try :
            a = list(map(lambda x : " ".join(x.split(delimeter)[i] for i in range(level)),listing)) #구분자로 자르고 ㅇ
            self[column] = a
            after = self
            return after
        except IndexError :
            print('레벨이 어울리지 않습니다.')

In [3]:
# 레벨트리 그리는 class
class LevelTree():
    def calculate(data, method, index, sep=""):
        global level_tmp
        sample = data.iloc[:, [index]] # 샘플로 10번째 행까지만 다룸
        
        sample.rename(columns = {data.columns[index] : 'base'}, inplace = True) # column 명을 우선 임의로 지정해서
        max_len = sample.base.map(lambda x: len(str(x))).max() # data들 중 가장 긴 길이의 데이터 값 저장
        
        sample.rename(columns = {'base' : 'Level 0'}, inplace = True) # 원활한 출력을 위해 level 0으로 다시 컬럼명 변경
        level = [''] # level[0]을 비우고 싶어서 ...(?)
        levelTree = sample # return 할 levelTree에 우선 level 0을 넣어 선언
        original = sample
        if method.find('마스킹') != -1: # 기법이 마스킹이면
            for i, j in zip(reversed(range(max_len)), range(1, max_len+1)): # i = 3(역순으로) / j = 1 에서 시작
                level.append(anonymous.masking(sample, 'Level 0', 'left', i, max_len))
                level[j].rename(columns = {'Level 0' : 'Level '+str(j) }, inplace = True) # column명 변경
                levelTree = pd.concat([levelTree, level[j]], axis=1) # levelTree에 산출한 레벨 이어붙이기
        elif method.find('총계처리') != -1:
            col = data.iloc[:, [index]]
            sample.loc[:, '평균'] = col[data.columns[index]].mean()
            sample.loc[:, '중앙값'] = col[data.columns[index]].median()
            sample.loc[:, '최솟값'] = col[data.columns[index]].min()
            sample.loc[:, '최댓값'] = col[data.columns[index]].max()
            # total.rename(columns = {'Level 0' : 'Level 1' }, inplace = True) # column명 변경
            # levelTree = pd.concat([levelTree, level['Level 1']], axis=1) # levelTree에 산출한 레벨 이어붙이기
        elif method.find('라운딩') != -1:
            if '.' not in str(original.loc[0,'Level 0']) : ## 정수 실수 구분하기 위한 . 찾기
                for i, j in zip(range(1,max_len), range(1, max_len)) :
                    temp = original.copy()
                    a = anonymous.Rounding(temp, 'Level 0', i , 0) 
                    a.rename(columns = {'Level 0' : 'Level '+str(j) }, inplace = True)
                    levelTree = pd.concat([levelTree,a],axis=1)
            elif '.' in str(original.loc[0,'Level 0']) : ## 실수 임을 나타냄
                for i, j in zip( range(-3,max_len-3), range(1, max_len+1) ) : ## 소수점 셋째자리부터 라운딩 시작.
                    temp = original.copy()
                    a = anonymous.Rounding(temp, 'Level 0', i , 0)
                    a.rename(columns = {'Level 0' : 'Level '+str(j) }, inplace = True)
                    levelTree = pd.concat([levelTree,a],axis=1)
        elif method.find('상하단코딩') != -1:
            for i,j in zip( reversed(range(4,10)), range(1,7) ):
                temp = original.copy()
                a = anonymous.Drange(temp, 'Level 0', i)
                a.rename(columns = {'Level 0' : 'Level '+str(j) }, inplace = True)
                levelTree = pd.concat([levelTree,a],axis=1)
        elif method.find('부분삭제') != -1 :
            lev = 1
            smth = 0
            for i in reversed(range(1,6)):
                temp = original.copy()
                a = anonymous.delete_partial(temp,'Level 0', i, sep)
                if len(a) == 0 : continue 
                smth +=1
                if smth == 1 : continue
                a.rename(columns = {'Level 0' : 'Level '+str(lev) }, inplace = True)
                levelTree = pd.concat([levelTree,a],axis=1)
                lev += 1
        level_tmp.append(levelTree)
        return levelTree

In [4]:
# 전처리 클래스 
class pretreatment():
    def value_normalize(after,column):   #수치형 데이터만 준비 완료 누를 때 이상치 제거 
            s = after[column]
            s = s[((s-s.mean())/s.std()>3) | ((s-s.mean())/s.std()<-3)]
            after = after[~after[column].isin(s)]

In [5]:
# 타입 변경 ui            
type_change_class = uic.loadUiType("type_change.ui")[0]
class TypeChangeWindow(QDialog, type_change_class) :
    def __init__(self) :
        super().__init__()
        self.setupUi(self)
        self.yes_button.clicked.connect(self.change)
        
    def change(self) :
        global type_to
        if self.radio_object.isChecked() == True :
            self.close()
            type_to = 'object'
        elif self.radio_int.isChecked() == True :
            self.close()
            type_to = 'int64'
        elif self.radio_float.isChecked() == True :
            self.close()
            type_to = 'float64'

In [6]:
# level 정하는 window
class LevelWindow(QWidget):
    col = ""
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        uic.loadUi('levelWindow_widget.ui', self)

        self.label_col.setText(self.col)

In [7]:
# delimiter 고를 수 있는 level window
class LevelWindow_partial(QWidget):
    col = ""
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        uic.loadUi('levelWindow_partial_widget.ui', self)
        
        self.label_col.setText(self.col)

In [8]:
def brute(*args): # 입력한 준식별자에 대해서 점수 Top3 매기기
    global top3
    
    # ISO / IEC 문서 참고하여 점수 측정
    eee = {'부분삭제' : {'S':0.5 ,'L':0.5,'I':0.5},
           '라운딩' : {'S':0 ,'L':0.5,'I':0.5},
           '마스킹' : {'S':1 ,'L':0.5,'I':0},
           '총계처리' : {'S':1 ,'L':1,'I':1},
           '상하단코딩' : {'S':0 ,'L':0.5,'I':0.5}
          }
        
    top3 = []
    bbb = list(product(*args, repeat = 1)) # 경우의수 (중복순열)
    list1 = []
        
    for i in bbb :
        sum1 = 0
        for j in i :
            sum1 +=sum(eee[j].values())
        list1.append(sum1 / len(type_combo_res)) # 해당 경우의 평균 점수를 list1에 저장
    
    for i in range(3) :
        top3.append(bbb[list1.index(max(list1))]) # list1에서 높은 점수 저장
        list1[list1.index(max(list1))] = 0 # 높은 점수 0으로 다음에 제외
    return top3 # 제일 높은 점수 3가지 경우 반환

In [9]:
def is_categorical(array_like): ## 카테고리형이면 true 반환
    return array_like.dtype.name == 'category'

In [10]:
def change_int(input): ## 상하단코딩 int값으로 변환하기 위함
    if '(' in input:
        return (float(str(input).strip('(]').split(', ')[0])+ float(str(input).strip('(]').split(', ')[1]))/2
    else :
        return 0

In [11]:
def K_anonymity(dataframe, listing, number): #df, 준식별 list, input K값,
    global same_size ## 짤리는 데이터에 0을 채워 넣을 데이터
    
    try:
        number = int(number) ## 만약 숫자를 입력하지 않는  다면 예외처리!
    except NameError:
        print("Input number value!")
        pass
    list_copy = [] ## 카테고리인 칼럼 저장하기
    for c in listing :
        if is_categorical(dataframe[c]): ## 카테고리인지 아닌지 함수구현
            dataframe[c] = dataframe[c].astype('str') ## 카테고리에 0을 넣기 위해 임시 형변환
            list_copy.append(c)
    ## list 는 준식별 정보를 담기 위해 for 문 사용 권장
    dataframe['count'] = dataframe.groupby(listing)[listing[0]].transform('size') ## list[0]으로 최소 칼럼 보장.
    
    #3중포문 ㅈㅅ 다른방법이 없는듯
    print(dataframe['count'])
    
    for a in dataframe.loc[dataframe['count']<number].index : #짤리는 데이터프레임의 index를 가져옴
        for b in dataframe.index :  # 원래 데이터프레임 index랑 비교
            if a == b : # 같으면
                for c in listing : # 준식별자로 입력된걸 0으로 바꿈
                    dataframe[c].loc[a] = 0     
    
    same_size = dataframe.copy()
    for i in list_copy: ## 상하단 코딩임을 알아야 하므로 다시 카테고리로 변환.
        same_size[i] = same_size[i].astype('category')
    
    dataframe = dataframe.loc[dataframe['count']>=number] # k 보다 높거나 같은 size만 추출함.
    dataframe_index = dataframe.index ## 인덱스가 초기화 되므로 일련번호 저장
    del dataframe['count']
    dataframe = dataframe.set_index(dataframe_index)
    dataframe = dataframe.sort_values(listing,ascending=True) ## k값으로 묶인 것을 보기위해서 sorting 과정 거친다.
    
    #dataframe에 K만족 못하는 데이터는 짤린다. 짤리는 데이터에 0을 채운게 same_size
    return dataframe

In [12]:
def Jaccard_similarity(doc1, doc2):
    # 두 집합의 자카르드 유사성이란 교집합의 크기를 합집합의 크기로 나눈 것이다.
    # 높을수록 좋음
    doc1 = set(doc1)
    #print('doc1 : ', doc1)
    doc2 = str(doc2)
    doc2 = set(doc2)
    #print('doc2 : ', doc2)
    #print('return len(doc1 & doc2) / len(doc1 | doc2) : ', len(doc1 & doc2) / len(doc1 | doc2))
    return len(doc1 & doc2) / len(doc1 | doc2)

In [13]:
def levenshtein(s1, s2, debug=False):
    # 작을수록 좋음
    s2 = str(s2)
    if len(s1) < len(s2):
        return levenshtein(s2, s1, debug)

    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))

        if debug:
            print(current_row[1:])

        previous_row = current_row

    return previous_row[-1]

In [14]:
def levenshtein(s1, s2, debug=False):
    # 작을수록 좋음
    s2 = str(s2)
    if len(s1) < len(s2):
        return levenshtein(s2, s1, debug)

    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))

        if debug:
            print(current_row[1:])

        previous_row = current_row

    return 1 - (previous_row[-1])

In [15]:
def mean_attr(X,Y,Z): # U1 : Mean Attribute (10%) # 평균값의 차이
    # 낮을수록 좋다
    # x=df1, y=df2, z=column
    a = X[Z][0]
    try:
            a == int(a) ## 만약 숫자를 입력하지 않는  다면 예외처리!
    except ValueError:
            print("Input number value!")
            pass
            return
        
    meanDiff=X[Z].mean()-Y[Z].mean()
    return 1-(abs(meanDiff/X[Z].mean()))
    #return abs(meanDiff)

In [16]:
def mean_corr(X,Y,Z): # U2 : Mean Correlation (10%)
    # 1에 가까울수록 좋음
    # x=df1, y=df2, z=column
    a = X[Z][0] ## 데이터 하나를 추출하여 int형으로 강제 형변환 한것과 비교,,, 같은 경우 정수형으로 보고 인식
    try:
            a == int(a) ## 만약 숫자를 입력하지 않는  다면 예외처리!
    except ValueError: ## 값이 일치하지 않을 경우
            print("Input number value!") ## 에러메세지 출력
            pass
            return ## NULL 값 반환(아무것도 반환 안함)
    df_corr = pd.DataFrame({'x': X[Z], 'y': Y[Z]}) ## KL과정을 거치기 전 df X와 거친 후 Y의 Z열(KL처리를 한 항목) 값을 받아서 데이터프레임 생성
    meanCorr=df_corr.corr(method='pearson') ## 피어슨 상관계수 계산 (df 자체적으로 돌아감)
    return abs(meanCorr.iloc[0,1]) ## 피어슨 상관계수 값 추출 0,1 즉 서로 다른 열을 계산한 값을 출력함,,,, 아래의 출력창 참고

In [17]:
def cos_sim(X, Y, Z): # U3 : Cosine Similarity (10%)
    # x=df1, y=df2, z=column
    a = X[Z][0]
    try:
            a == int(a) ## 만약 숫자를 입력하지 않는  다면 예외처리!
    except ValueError:
            print("Input number value!")
    meanSim = dot(X[Z],Y[Z])/(norm(X[Z])*norm(Y[Z]))    
    return meanSim

In [18]:
def info_loss(X, Y,Z): # U4 : Information Loss (15%)
    # 지정된 특정 칼럼(속성)에 대한 정형화된 차이값들의 평균
    # X.shape = Y.shape # X는 데이터프레임 Y도 데이터프레임 Z 는 컬럼 리스트로 받아야할듯(수치형)
    a = ((abs(X[Z]-Y[Z]))/X[Z]).sum()
    return 1 - (a / len(Y[Z].index))

In [19]:
class StdoutRedirect(QObject):
    printOccur = pyqtSignal(str, str, name="print")
 
    def __init__(self, *param):
        QObject.__init__(self, None)
        self.daemon = True
        self.sysstdout = sys.stdout.write
        #self.sysstderr = sys.stderr.write
 
    def stop(self):
        sys.stdout.write = self.sysstdout
        #sys.stderr.write = self.sysstderr
 
    def start(self):
        sys.stdout.write = self.write
        #sys.stderr.write = lambda msg : self.write(msg, color="red")
 
    def write(self, s, color="black"):
        sys.stdout.flush()
        self.printOccur.emit(s, color)

In [20]:
form_class = uic.loadUiType("log.ui")[0]

class ConsoleView(QWidget, form_class):
    def __init__(self):
        super().__init__()
        self.setupUi(self)
        
        # member variable     
        self._stdout = StdoutRedirect()
        self._stdout.start()
        self._stdout.printOccur.connect(lambda x : self._append_text(x))
 
    def _append_text(self, msg):
        self.textBrowser.moveCursor(QtGui.QTextCursor.End)
        self.textBrowser.insertPlainText(msg)
        # refresh textedit show, refer) https://doc.qt.io/qt-5/qeventloop.html#ProcessEventsFlag-enum
        QApplication.processEvents(QEventLoop.ExcludeUserInputEvents)


In [21]:
# 비식별 적용 버튼 클릭 시, 새 창 띄워짐 (= anonymousWindow)
anonymous_class = uic.loadUiType("anonymousWindow_2.ui")[0]

class AnonymousWindow(QDialog, anonymous_class):
    def __init__(self) :
        super().__init__()
        self.setupUi(self)
        self.checked_method = [] # 기법 체크박스 값 리스트
        self.combo_sep = [] # 구분자 담는 리스트
        self.next_index = 2 # stackedWidget의 페이지 수 
        self.typelist = 0 # 있어 이런거..
        self.method = [] # methodBox에서 사용할 비식별 기법 리스트
        self.level = []
        
        # 준식별자로 택한 column index
        global type_combo_res
        type_combo_res = np.array(type_combo_res) # list에서 해당 단어가 포함된 모든 index를 얻기 위해 numpy 사용
        self.QI_index = np.where(type_combo_res == '준식별자')[0] # 준식별자인 컬럼 index 리스트

        self.lineEdit_k.setValidator(QIntValidator(1,100,self)) # k 입력 받는 lineEdit => 1~100 정수만

        # 버튼 이벤트
        self.cancel_button.clicked.connect(self.closeWindow) # 취소 버튼 클릭 시, 창 닫힘
        self.next_button.clicked.connect(self.go_levelWindow) # next 버튼 클릭 시, addWidget한 위젯으로 이동할 수 있는 함수 호출
        
        # Window 이름 지정
        self.setWindowTitle("anonymous setting")
        
        # groupBox list 항목 -> horiziontal Layout 에 추가
        self.method_box_list = self.methodBox() # methodBox 함수를 이용해 groupbox 리스트를 받아옴
        for i in self.method_box_list :
            self.horizontalLayout_groupbox.addWidget(i) # groupbox list 하나씩 horizontalLayout에 addWidget 함
        
        self.show() #test2
        
    def closeWindow(self):
        self.close()
    
    def methodBox(self): # 비식별 기법 체크박스 생성
        global data
        groupbox = [] # 준식별자에 적용할 checkbox의 groupbox를 담는 리스트
        j = 0 # groupbox index로 사용할 변수
        brutebox = []
        
        for i in self.QI_index:
            if data[data.columns[i]].dtype == 'object' :
                brutebox.append(['부분삭제', '마스킹'])
            elif data[data.columns[i]].dtype == 'int64' or 'float64' :
                brutebox.append(['부분삭제', '라운딩', '마스킹', '총계처리'])
            elif data[data.columns[i]].dtype == 'category' :
                brutebox.append(['부분삭제', '마스킹'])
                
            groupbox_objName = "groupBox" + str(i)
            groupbox.append(QGroupBox(data.columns[i] + " (준식별자)", objectName=groupbox_objName)) # groupbox 제목 지정해서 groupbox list에 담음
            
            self.method = ['부분삭제', '라운딩', '마스킹', '총계처리', '상하단코딩'] # 비식별 기술 리스트 
            chkbox_list = [] # 각 groupbox에 들어갈 checkbox 리스트

            combo = QComboBox(objectName="aa")
            combo.addItems(["띄어쓰기", "-", ";", "|", "@"])
            combo.setFixedSize(100,20)
            combo.setStyleSheet('margin-left : 20')
            
            check_idx = 0
            for item in self.method:
                checkbox_objName = "checkBox_" + str(i) + "_" + str(check_idx) 
                if data[data.columns[i]].dtype == 'object':
                    if item in ['부분삭제','마스킹']:
                        chkbox_list.append(QRadioButton(item, objectName=checkbox_objName)) # 체크박스 리스트에 각 체크박스 항목을 넣는 for문
                        if item.find('부분삭제') != -1:
                            combo_objName = "combo_" + str(i)
                            combo = QComboBox(objectName=combo_objName)
                            combo.addItems(["띄어쓰기", "-", ";", "|", "@"])
                            combo.setFixedSize(100,20)
                            combo.setStyleSheet('margin-left : 20')
                            chkbox_list.append(combo)
                    else:
                        chkbox_list.append(QRadioButton(item, objectName=checkbox_objName).setDisabled(True))
                        
                else:
                    if item in ['라운딩','마스킹','총계처리' ,'상하단코딩','부분삭제']:
                        chkbox_list.append(QRadioButton(item, objectName=checkbox_objName)) # 체크박스 리스트에 각 체크박스 항목을 넣는 for문
                        if item.find('부분삭제') != -1:
                            combo_objName = "combo_" + str(i)
                            combo = QComboBox(objectName=combo_objName)
                            combo.addItems(["띄어쓰기", "-", ";", "|", "@"])
                            combo.setFixedSize(100,20)
                            combo.setStyleSheet('margin-left : 20')
                            chkbox_list.append(combo)
                    else:
                        chkbox_list.append(QRadioButton(item, objectName=checkbox_objName).setDisabled(True))
                        
                check_idx += 1
            vbox = QVBoxLayout() 
            for chkbox in chkbox_list:
                vbox.addWidget(chkbox) # box layout에 체크박스들을 담은 리스트를 addWidget함
            groupbox[j].setLayout(vbox)
            groupbox[j].setLayout
            j+=1 # groupbox index 증가
            
        brute(*brutebox) # 비식별 추천
        
        rank1 = ""
        for i, j in zip(range(len(self.QI_index)), range(1, len(self.QI_index)+1)):
            tmp = str(top3[0][i])
            rank1 += tmp
            if i != len(self.QI_index)-1: rank1 += " / "

        rank2 = ""
        for i, j in zip(range(len(self.QI_index)), range(1, len(self.QI_index)+1)):
            tmp = str(top3[1][i])
            rank2 += tmp
            if i != len(self.QI_index)-1: rank2 += " / "

        rank3 = ""
        for i, j in zip(range(len(self.QI_index)), range(1, len(self.QI_index)+1)):
            tmp = str(top3[2][i])
            rank3 += tmp
            if i != len(self.QI_index)-1: rank3 += " / "

        self.label_2.setText("TOP 1 : " + rank1)
        self.label_3.setText("TOP 2 : " + rank2)
        self.label_4.setText("TOP 3 : " + rank3)
        return groupbox
    
    def go_levelWindow(self):
        global data
        global after
        global k
        global oriental
        global same_size
        
        if self.typelist == 0:
            self.getTypeList() # getTypeList 함수는 한 번만 실행되게 !
            k = self.lineEdit_k.text() # k 입력받음
            self.typelist += 1
            
        if self.next_index > 2 :
            self.level.append(self.stackedWidget.currentWidget().tableWidget.currentColumn())
            self.make_after(self.next_index-3, self.stackedWidget.currentWidget().tableWidget.currentColumn()) # 선택한 level로 after를 만드는 함수
        
        if self.next_index == 2+len(self.QI_index):
            abc = [] # 준식별자로 선택된 칼럼 이름 저장
            for i in range(len(type_combo_res)) :
                if type_combo_res[i] == '준식별자' :
                    abc.append(after.columns[i])
            oriental = after.copy()
            
            print("****** 적용한 비식별 기법 ******")
            for i, j in zip(self.QI_index, range(len(self.QI_index))):
                print("[준식별자] ", data.columns[i], " : ", self.checked_method[j], " ( 레벨 : ", self.level[j], ")")
            
            after = K_anonymity(after, abc, k)
            print()
            print("****** ", k, "- 익명성 적용 결과 ******")
            print('전체 레코드 :', len(data))
            print('K-익명성 처리 되어 손실된 레코드 : ',len(data)-len(after))
            print()
            same = same_size.copy()
            da = data.copy()
            
            is_ok = 0
            
            for a in abc :
                if same[a].dtype == 'object' : # 문자열 유용성 ###
                    sum_jaccard = 0
                    sum_levenshtein =0
                    for i in range(len(same.index)):
                        sum_jaccard+=Jaccard_similarity(da[a][i],same[a][i])
                        sum_levenshtein+=levenshtein(da[a][i],same[a][i],debug=False)/len(da[a][i])
                    print('자카르 : ', sum_jaccard/len(same.index))
                    print('레벤슈타인 : ', sum_levenshtein/len(same.index))
                    if (sum_jaccard/len(same.index) + sum_levenshtein/len(same.index)) / 2 >= 0.7 :
                        is_ok += 1
                else : # 수치형 유용성
                    if is_categorical(same[a]): ## 만약 카테고리형 이면 
                        same[a] = same[a].astype('str') ## 카테고리 계산을 위해 형변환
                        same[a] = same[a].apply(change_int) ## 그리고 중앙값으로 치환
                    print('평균 차이 : ', mean_attr(da, same , a))
                    print('피어슨 차이 : ', mean_corr(da, same, a))
                    print('코사인 차이 : ', cos_sim(da, same, a))
                    print('info_loss : ', info_loss(da,same,a))
                    if (mean_attr(da, same , a) + mean_corr(da, same, a) + cos_sim(da, same, a) + info_loss(da,same,a)) / 4 >= 0.5 :
                        is_ok += 1
                if is_ok == 2 :
                    print('좋아 합격!')
        """ 
            - stackedWidget 추가하는 과정
             참고 ! index 0 : 기본 k 입력 받고 비식별 기법 적용하는 위젯
                    index 1 : 빈칸
                    index 2 : addWidget 한 내용
        """
        for method, qi, i in zip(self.checked_method, self.QI_index, range(len(self.QI_index))):
            self.levelWindow = LevelWindow()
            self.levelWindow.label_col.setText(data.columns[qi]) # 레벨트리 화면 제목 지정
            if method.find('부분삭제') != -1: # 부분삭제 비식별기법 사용 시
                level = LevelTree.calculate(data, method, qi, self.combo_sep[i])
            else:
                level = LevelTree.calculate(data, method, qi)
            self.levelWindow.tableWidget.setRowCount(preview) # data type, 개인정보 유형 -> "2행" 이후 데이터 출력
            self.levelWindow.tableWidget.setColumnCount(len(list(level)))
            self.levelWindow.tableWidget.setHorizontalHeaderLabels(list(level))

            for i in range(preview) :
                for j in range (len(list(level))) :
                     self.levelWindow.tableWidget.setItem(i, j, QTableWidgetItem(format(level.values.tolist()[i][j])))
            self.stackedWidget.addWidget(self.levelWindow)

        if self.next_index == 1+len(self.QI_index): # 마지막 stackedWidget일 때, 
            self.next_button.setText('완료') # 다음 버튼 대신 완료 버튼으로 변경
            self.next_button.clicked.connect(self.complete) # 완료 버튼 클릭 시, complete 함수 실행
                    
        # stackedWidget index 이동
        #print(self.next_index,"번째 stackedWidget")
        self.stackedWidget.setCurrentIndex(self.next_index)
        
        self.next_index += 1 # 다음버튼 클릭 후 그 다음에 다음버튼할 index
    
    def make_after(self, i, level): # i : 몇 번째 수행인지, level : 선택한 level이 무엇인지
        global after
        global level_tmp

        qi = self.QI_index[i]
        level_df = level_tmp[i]
        
        #print("make_after qi : ", qi)
        #print("make_after level : ",level)
        
        after.iloc[:, qi] = level_df.iloc[:, level]
                    
    def complete(self): # 완료 버튼 클릭 시
        #WindowClass.print_after(WindowClass)
        self.close()
    
    def getTypeList(self): # 체크된 기법 리스트에 담는 과정     
        for i in self.QI_index: # method_box_list의 모든 그룹 박스
            for j in range(len(self.method)): # 체크박스 (0~5 = range(6))
                checkbox_name = "checkBox_" + str(i) + "_" + str(j)
                chkbox = self.findChild(QRadioButton, checkbox_name)
                try:
                    if chkbox.isChecked() : self.checked_method.append(chkbox.text()) 
                except:
                    pass
            combo_name = "combo_" + str(i)
            cmb = self.findChild(QComboBox, combo_name)
            self.combo_sep.append(cmb.currentText())
        #print("checked_method : ", self.checked_method)
        #print("QI_index : ", self.QI_index)

In [22]:
form_class = uic.loadUiType("4thUI2.ui")[0]

#화면을 띄우는데 사용되는 Class 선언
class WindowClass(QMainWindow, form_class) :    
    def __init__(self) :
        super().__init__()
        self.setupUi(self)
        
        # 로고 이미지
        qPixmapVar = QPixmap()
        self.Logo.setPixmap(QPixmap("../인터페이스/logo.png"))
        
        # 버튼 이벤트
        self.log_button.clicked.connect(self.log_print) # label 버튼 클릭관련
        self.open_button.clicked.connect(self.open_file) # open_button 클릭 관련
        self.apply_button.clicked.connect(self.open_anonymousWindow) # '비식별 처리' 버튼 클릭 관련
        self.save_button.clicked.connect(self.save_file) # save_button 클릭 관련
        self.type_button.clicked.connect(self.type_change) # 타입 변경
        self.set_button.clicked.connect(self.set_after) # --
        
        self.log_button.setStyleSheet('image:url(../인터페이스/log.png); border:0px;')
        self.open_button.setStyleSheet('image:url(../인터페이스/open.png); border:0px;')
        self.type_button.setStyleSheet('image:url(../인터페이스/type.png); border:0px;')
        self.set_button.setStyleSheet('image:url(../인터페이스/set.png); border:0px;')
        self.apply_button.setStyleSheet('image:url(../인터페이스/anonymous.png); border:0px;')
        self.save_button.setStyleSheet('image:url(../인터페이스/save.png); border:0px;')

        self.loc = 0
        self.job = 0
        loc = self.loc
    
    def log_print(self):
        log = ConsoleView()
        log.show()
        
    def open_file(self) : # 파일 불러오기
        global data
        global count
        global info_header
        
        open_name = QFileDialog.getOpenFileName(self, 'Open File', './')       
        if (max(open_name) != ""):
            data = pd.read_csv(open_name[0], encoding='utf-8')
            self.status.setText(open_name[0]+" 파일이 열렸습니다.")
            self.status.setAlignment(Qt.AlignCenter)

            global after # 비식별 처리 후 데이터를 저장할 전역변수 after
            after = data.copy()
            count = list()
            
            # 개인정보 유형(식별자, 준식별자, 민감정보, 비민감정보) Combo Box
            global type_combo_list # 개인정보 유형 콤보박스들을 담는 리스트
            type_combo_list = []
            for i in range(len(data.columns)):
                type_combo = QComboBox()
                type_combo.addItems(['비민감정보','식별자', '준식별자', '필요없는칼럼'])
                type_combo_list.append(type_combo)
            
            # row header 추가하기 위한 list 생성
            row = [*range(1, preview + 2)]
            row.insert(0,"타입")
            row.insert(0,"")
            row = list(map(str, row))
            
            # table_1 widget 행, 열 추가
            self.table_1.setRowCount(preview + 2) # data type, 개인정보 유형 -> "2행" 이후 데이터 출력
            self.table_1.setColumnCount(len(data.columns))
            self.table_1.setHorizontalHeaderLabels(list(data))
            self.table_1.setVerticalHeaderLabels(row)

            # table_1 widget에 개인정보 유형 콤보박스 추가
            for i in range(len(data.columns)):
                self.table_1.setCellWidget(0, i, type_combo_list[i]) # 0번째 행에 개인정보 유형 Combo Box
            
            # 비식별 전 table_1에 
            # table_3 widget에 info추가 하기 위해서 수치형 개수 count
            for i in range(len(data.columns)) :
                if data[data.columns[i]].dtype == 'object' :
                    self.table_1.setItem(1, i, QTableWidgetItem('문자열')) # 1번째 행에 데이터 타입
                elif data[data.columns[i]].dtype == 'int64' :
                    self.table_1.setItem(1, i, QTableWidgetItem('정수')) # 1번째 행에 데이터 타입
                elif data[data.columns[i]].dtype == 'float64' :
                    self.table_1.setItem(1, i, QTableWidgetItem('실수')) # 1번째 행에 데이터 타입
                
                if (data[data.columns[i]].dtype == 'int64') | (data[data.columns[i]].dtype == 'float64') :
                    count.append(i)

            # table_1 widget에 불러온 파일의 데이터 추가
            for i in range (preview) :
                for j in range (len(list(data))) :
                    self.table_1.setItem(i + 2, j, QTableWidgetItem(format(data.values.tolist()[i][j]))) # 개인정보 유형, 데이터타입 -> "2행" 이후 데이터 출력
            
            # 수치형 데이터 table_3에 min, max ,median 등
            self.table_3.setRowCount(len(count))
            self.table_3.setColumnCount(len(list(info_header)))
            self.table_3.setHorizontalHeaderLabels(list(info_header))
            
            # 수치형 데이터의 이름, 최솟값, 최댓값, 평균, 중앙값, 표준편차, 최빈값 출력
            for i in range(len(count)) :
                self.table_3.setItem(i, 0, QTableWidgetItem(format(data.columns[count[i]])))
                self.table_3.setItem(i, 1, QTableWidgetItem(format(data[data.columns[count[i]]].min())))
                self.table_3.setItem(i, 2, QTableWidgetItem(format(data[data.columns[count[i]]].max())))
                self.table_3.setItem(i, 3, QTableWidgetItem("%.2lf"%(data[data.columns[count[i]]].mean())))
                self.table_3.setItem(i, 4, QTableWidgetItem(format(data[data.columns[count[i]]].median())))
                self.table_3.setItem(i, 5, QTableWidgetItem("%.3lf"%(data[data.columns[count[i]]].std())))
                self.table_3.setItem(i, 6, QTableWidgetItem(format(data[data.columns[count[i]]].value_counts().index[0])))
        else:
            print('파일 선택 안함')
            
    # 타입 변경
    def type_change(self) :
        global type_to
        global data
        
        type_change_Window = TypeChangeWindow()
        type_change_Window.exec_()
        column = self.table_1.currentColumn()
        data[data.columns[column]] = data[data.columns[column]].astype(type_to)
        for i in range(len(data.columns)) :
                if data[data.columns[i]].dtype == 'object' :
                    self.table_1.setItem(1, i, QTableWidgetItem('문자열')) # 1번째 행에 데이터 타입
                elif data[data.columns[i]].dtype == 'int64' :
                    self.table_1.setItem(1, i, QTableWidgetItem('정수')) # 1번째 행에 데이터 타입
                elif data[data.columns[i]].dtype == 'float64' :
                    self.table_1.setItem(1, i, QTableWidgetItem('실수')) # 1번째 행에 데이터 타입
    
    def set_after(self) :
        global type_combo_list
        global type_combo_res
        global count
        global info_header
        
        type_combo_res = [] # 개인정보 유형 콤보박스 선택 값 넣는 리스트
        for combo in type_combo_list:
            type_combo_res.append(combo.currentText()) # 현재 콤보박스 text를 type_combo_res에 append

        # 수치형 전처리 
        for i in range(len(after.columns)):
            if (type_combo_res[i] == '식별자') or (type_combo_res[i] =='필요없는칼럼') :
                after[after.columns[i]] = ''
            if(after[after.columns[i]].dtype != object):
                pretreatment.value_normalize(after, after.columns[i])
            else:
                pass
        
        # row header 추가하기 위한 list 생성
        row = [*range(1, preview + 1)]
        row.insert(0,"타입")
        row = list(map(str, row))
        
        self.table_2.setRowCount(preview + 1) # data type, 개인정보 유형 -> "2행" 이후 데이터 출력
        self.table_2.setColumnCount(len(list(after)))
        self.table_2.setHorizontalHeaderLabels(list(after))
        self.table_2.setVerticalHeaderLabels(row)

        # 수치형 데이터 table_3에 min, max ,median 등
        self.table_4.setRowCount(len(count))
        self.table_4.setColumnCount(len(list(info_header)))
        self.table_4.setHorizontalHeaderLabels(list(info_header))
        
        # 현재 상태 창 텍스트 변경
        self.status.setText("비식별 처리할 준비가 완료되었습니다.")
        self.status.setAlignment(Qt.AlignCenter)
        
        self.print_after() # table 내용은 print_after에서 출력
        
    def print_after(self):        
        self.table_2.clearContents()
        self.table_4.clearContents()
        global oriental
        global count
        global preview
        global after
        
        # table_2 widget에 불러온 파일의 데이터 추가
        for i in range (preview) :
                for j in range (len(list(after))) :
                    if i>=len(after): break # preview 값(10)보다 after의 길이가 더 작을 경우 for문 빠져나옴
                    self.table_2.setItem(i + 1, j, QTableWidgetItem(format(after.values.tolist()[i][j]))) # 개인정보 유형, 데이터타입 -> "2행" 이후 데이터 출력

        # table_2 widget에 데이터 타입 추가
        for i in range(len(after.columns)) :
                if after[after.columns[i]].dtype == 'object' :
                    self.table_2.setItem(0, i, QTableWidgetItem('문자열')) # 1번째 행에 데이터 타입
                elif after[after.columns[i]].dtype == 'int64' :
                    self.table_2.setItem(0, i, QTableWidgetItem('정수')) # 1번째 행에 데이터 타입
                elif after[after.columns[i]].dtype == 'float64' :
                    self.table_2.setItem(0, i, QTableWidgetItem('실수')) # 1번째 행에 데이터 타입
                    
        # 수치형 데이터의 이름, 최솟값, 최댓값, 평균, 중앙값, 표준편차, 최빈값 출력
        for i in range(len(count)) :
            if(after[after.columns[count[i]]].dtype == 'int64' or after[after.columns[count[i]]].dtype == 'float64'):
                self.table_4.setItem(i, 0, QTableWidgetItem(format(after.columns[count[i]])))
                self.table_4.setItem(i, 1, QTableWidgetItem(format(after[after.columns[count[i]]].min())))
                self.table_4.setItem(i, 2, QTableWidgetItem(format(after[after.columns[count[i]]].max())))
                self.table_4.setItem(i, 3, QTableWidgetItem("%.2lf"%(after[after.columns[count[i]]].mean())))
                self.table_4.setItem(i, 4, QTableWidgetItem(format(after[after.columns[count[i]]].median())))
                self.table_4.setItem(i, 5, QTableWidgetItem("%.3lf"%(after[after.columns[count[i]]].std())))
                self.table_4.setItem(i, 6, QTableWidgetItem(format(after[after.columns[count[i]]].value_counts().index[0])))
            else:
                pass


            
    def open_anonymousWindow(self):
        # 준식별자 값 전달 필요
        anonymousWindow = AnonymousWindow()
        anonymousWindow.exec_()
        #anonymousWindow.show() #test1
        
        self.print_after() # anonymous Window에서 비식별 적용 후, print after 다시 수행
        
        # 현재 상태 창 텍스트 변경
        self.status.setText("비식별 기법이 적용되었습니다.")
        self.status.setAlignment(Qt.AlignCenter)
                
    def save_file(self) : # 비식별처리 후 파일 저장
        global same_size
        
        save_name = QFileDialog.getSaveFileName(self, 'Open File', './', "CSV Files(*.csv);; All Files(*)")
        after.to_csv(save_name[0], encoding='utf-8', index=False)
        # 현재 상태 창 텍스트 변경
        self.status.setText(save_name[0]+" 파일으로 저장되었습니다.")
        self.status.setAlignment(Qt.AlignCenter)
        
if __name__ == "__main__" :
    app = QApplication(sys.argv) 
    myWindow = WindowClass()
    myWindow.show()
    app.exec_()

C:\Users\Feel\anaconda3\lib\site-packages\pandas\core\frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
<ipython-input-2-1e9f4b84d576>:41: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  self.backup = self[column]
C:\Users\Feel\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
